In [1]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime


In [2]:
# Load known faces using OpenCV's Haar Cascades and encodings using LBPH
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Directory where known faces are stored
faces_dir = 'faces'
labels = []
faces = []
label_id = 0
label_dict = {}

# Process each face in the 'faces' directory
for filename in os.listdir(faces_dir):
    image_path = os.path.join(faces_dir, filename)
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces_in_image = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces_in_image:
        roi_gray = gray_image[y:y+h, x:x+w]
        faces.append(roi_gray)
        label_dict[label_id] = filename.split('_')[0]
        labels.append(label_id)
    label_id += 1

recognizer.train(faces, np.array(labels))

# Initialize some variables
face_locations = []
face_names = []

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame=cv2.flip(frame,1)
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_locations = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    face_names = []
    for (x, y, w, h) in face_locations:
        roi_gray = gray_frame[y:y+h, x:x+w]
        label, confidence = recognizer.predict(roi_gray)

        if confidence < 100:  # Confidence threshold for recognizing face
            name = label_dict[label]
        else:
            name = "Unknown"
        
        face_names.append((name, x, y, w, h))

        # Mark attendance
        with open('attendance.csv', 'a') as f:
            writer = csv.writer(f)
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([name, dt_string])

    # Display the results
    for (name, x, y, w, h) in face_names:
        # Draw a box around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw a label with a name below the face
        #cv2.rectangle(frame, (x, y + h - 28), (x + w, y + h), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (x+25 , y-5 ), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()
